In [1]:
import json
import pandas as pd

# Read the JSON file
with open('quotes.json', 'r', encoding='utf-8') as file:
    quotes = json.load(file)

df = pd.DataFrame(quotes)

print(df.head())


                                               Quote          Author  \
0  Don't cry because it's over, smile because it ...       Dr. Seuss   
1  Don't cry because it's over, smile because it ...       Dr. Seuss   
2  I'm selfish, impatient and a little insecure. ...  Marilyn Monroe   
3  I'm selfish, impatient and a little insecure. ...  Marilyn Monroe   
4  I'm selfish, impatient and a little insecure. ...  Marilyn Monroe   

                                                Tags  Popularity   Category  
0  [attributed-no-source, cry, crying, experience...    0.155666       life  
1  [attributed-no-source, cry, crying, experience...    0.155666  happiness  
2  [attributed-no-source, best, life, love, mista...    0.129122       love  
3  [attributed-no-source, best, life, love, mista...    0.129122       life  
4  [attributed-no-source, best, life, love, mista...    0.129122      truth  


In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48391 entries, 0 to 48390
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Quote       48391 non-null  object 
 1   Author      48391 non-null  object 
 2   Tags        48391 non-null  object 
 3   Popularity  48391 non-null  float64
 4   Category    48391 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.8+ MB
None


In [ ]:
import random
import pandas as pd

class DetailedQuoteGenerator:
    def __init__(self, df):
        self.df = df
        self.authors = df['Author'].unique()
        self.tags = [tag for tags in df['Tags'] for tag in tags]
        self.categories = df['Category'].unique()

    def generate_quote(self):

        # Randomly select a row from the DataFrame
        quote_row = self.df.sample(n=1).iloc[0]

        # Generate a new quote using Markov Chain-like approach
        words = quote_row['Quote'].split()
        new_quote = self.generate_markov_quote(words)

        # Use the selected row's author and category
        author = quote_row['Author']
        category = quote_row['Category']

        # Randomly select one tag from the quote's tags
        tag = random.choice(quote_row['Tags'])

        # Generate a random popularity score
        popularity = random.randint(1, 100)

        return {
            'Quote': new_quote,
            'Author': author,
            'Tags': tag,
            'Popularity': popularity,
            'Category': category
        }

    def generate_markov_quote(self, words, max_length=30):
        markov_dict = {}
        for i in range(len(words) - 1):
            if words[i] not in markov_dict:
                markov_dict[words[i]] = []
            markov_dict[words[i]].append(words[i + 1])

        current_word = random.choice(words)
        result = [current_word]

        while len(result) < max_length:
            if current_word not in markov_dict:
                break
            next_word = random.choice(markov_dict[current_word])
            result.append(next_word)
            current_word = next_word

        return ' '.join(result)

# Assuming 'df' is your DataFrame from the previous step
generator = DetailedQuoteGenerator(df)
new_quote = generator.generate_quote()
print(f"Quote: {new_quote['Quote']}")
print(f"Author: {new_quote['Author']}")
print(f"Tag: {new_quote['Tags']}")
print(f"Popularity: {new_quote['Popularity']}")
print(f"Category: {new_quote['Category']}")
print()


Quote: works of art.
Author: Claude Debussy
Tag: Works
Popularity: 82
Category: arts

Quote: as I've gotten older, because I was never one of the part.
Author: Michael Caine
Tag: Mirror
Popularity: 32
Category: success

Quote: happy ending, only the moment.
Author: Ernst Jünger,  The Glass Bees
Tag: happiness 
Popularity: 80
Category: happiness

Quote: They tend to love the future.
Author: Richard Rohr,  Falling Upward: A Spirituality for the Two Halves of Life
Tag: religion
Popularity: 53
Category: religion

Quote: God, suggests my Guru. Look for God, suggests my Guru. Look for God, suggests my Guru. Look for God, suggests my Guru. Look for water.
Author: Elizabeth Gilbert
Tag: god
Popularity: 87
Category: god



In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import random
import matplotlib.pyplot as plt
import os

# Configure TensorFlow to use GPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU is available and configured for use")
else:
    print("No GPU found. Using CPU.")
    
class DeepLearningQuoteGenerator:
    def __init__(self, df, seq_length=40, batch_size=128, epochs=1):
        self.df = df
        self.seq_length = seq_length
        self.batch_size = batch_size
        self.epochs = epochs
        self.chars = sorted(list(set(''.join(df['Quote']))))
        self.char_to_idx = {char: idx for idx, char in enumerate(self.chars)}
        self.idx_to_char = {idx: char for idx, char in enumerate(self.chars)}
        self.vocab_size = len(self.chars)
        self.model = self.build_model()
        self.history = None

    def build_model(self):
        model = Sequential([
            LSTM(256, input_shape=(self.seq_length, self.vocab_size), return_sequences=True),
            Dropout(0.2),
            LSTM(256),
            Dropout(0.2),
            Dense(self.vocab_size, activation='softmax')
        ])
        model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001))
        return model

    def data_generator(self):
        text = ' '.join(self.df['Quote'])
        while True:
            for i in range(0, len(text) - self.seq_length, self.batch_size):
                X = np.zeros((self.batch_size, self.seq_length, self.vocab_size))
                y = np.zeros((self.batch_size, self.vocab_size))
                for j in range(self.batch_size):
                    if i + j + self.seq_length < len(text):
                        seq = text[i+j:i+j+self.seq_length]
                        next_char = text[i+j+self.seq_length]
                        for t, char in enumerate(seq):
                            X[j, t, self.char_to_idx[char]] = 1
                        y[j, self.char_to_idx[next_char]] = 1
                yield X, y

    def train(self):
        steps_per_epoch = len(' '.join(self.df['Quote'])) // self.batch_size
        self.history = self.model.fit(self.data_generator(), 
                                      steps_per_epoch=steps_per_epoch, 
                                      epochs=self.epochs,
                                      verbose=1)

    def generate_quote(self, seed_text, length=100):
        generated = seed_text
        for _ in range(length):
            x = np.zeros((1, self.seq_length, self.vocab_size))
            for t, char in enumerate(generated[-self.seq_length:]):
                if char in self.char_to_idx:
                    x[0, t, self.char_to_idx[char]] = 1
            
            preds = self.model.predict(x, verbose=0)[0]
            next_index = np.random.choice(len(preds), p=preds)
            next_char = self.idx_to_char[next_index]
            generated += next_char
            
            if next_char == '.':
                break
        
        return generated

    def plot_training_results(self):
        plt.figure(figsize=(12, 6))
        plt.plot(self.history.history['loss'])
        plt.title('Model Loss During Training')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.show()

    def save_model(self, filepath):
        self.model.save(filepath)
        print(f"Model saved to {filepath}")

    @classmethod
    def load_model(cls, filepath, df):
        generator = cls(df)
        generator.model = load_model(filepath)
        print(f"Model loaded from {filepath}")
        return generator

# Create and train the generator
generator = DeepLearningQuoteGenerator(df)
generator.train()

# Plot the training results
generator.plot_training_results()

# Save the model
generator.save_model('quote_generator_model.h5')

# Generate some quotes
seed = random.choice(df['Quote'])[:40]  # Use the first 40 characters of a random quote as seed
new_quote = generator.generate_quote(seed)
print(f"Generated Quote: {new_quote}")
print()


No GPU found. Using CPU.
13845/51992 ━━━━━━━━━━━━━━━━━━━━ 1:59:29 188ms/step - loss: 2.2956

KeyboardInterrupt: 